In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_next_27_hour,rides_next_28_hour,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour
0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,10.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,1.0,2.0,5.0,2.0,2.0,2.0
2,0.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,5.0,4.0,1.0,8.0
3,4.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,4.0,...,2.0,1.0,1.0,0.0,1.0,1.0,7.0,2.0,2.0,3.0
4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
111212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,0.0
111213,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0
111214,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date = datetime(2022, 6, 1, 0 , 0, 0),
    targets_columns_names= [c for c in df.columns if  c.startswith('rides_next')]
    )
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')


X_train.shape=(40713, 674)
y_train.shape=(40713, 36)
X_test.shape=(70503, 674)
y_test.shape=(70503, 36)


In [4]:
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]


In [6]:
import xgboost as xgb

In [7]:
model = xgb.XGBRegressor()
model.fit(X_train_only_numeric, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [8]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

array([[ 1.5601035 ,  1.3401357 ,  0.02426209, ...,  1.5457414 ,
         1.9311773 ,  2.8536594 ],
       [ 3.7590663 ,  1.4488322 , -0.07925513, ...,  2.5916474 ,
         4.908646  ,  2.854332  ],
       [ 2.4798326 ,  1.2453306 ,  0.37254122, ..., -0.20751803,
         1.2950553 ,  0.9405526 ],
       ...,
       [ 0.36880848,  0.26842114,  0.21617861, ...,  0.45184943,
         0.53920954,  0.33215246],
       [ 0.7097338 ,  0.08962848,  0.2195925 , ...,  0.7319126 ,
         0.36206734,  0.5940909 ],
       [ 0.552232  ,  0.38717625,  0.6991058 , ...,  0.7885607 ,
         0.6633604 ,  0.13949385]], dtype=float32)

In [9]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=}')

test_mae=0.7072761
